In [2]:
import re
import requests
from bs4 import BeautifulSoup
import json

In [11]:
article_dict = list()
counts = 0 
for i in range(1, 38):
    response = requests.get(f'https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100')
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')  
    else:
        print(f"Error: Unable to fetch the content:{response.status_code}")

    all_article_in_a_page = soup.find_all('article', {"itemprop": "review"})
    pattern = r'\((.*?)\)'
    get_int_pattern = r'\d+'
    
    for i, article in enumerate(all_article_in_a_page):
        table = article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'review-stats'}).find('table').find_all('tr')
        
        review_dict = {}
        for tr in table:
            header_td = tr.find('td', class_='review-rating-header')
            value_td = tr.find('td', class_='review-value')
            stars_td = tr.find('td', {'class': 'review-rating-stars'})
            
            if header_td and value_td:
                header = header_td.get_text(strip=True)
                value = value_td.get_text(strip=True)
                review_dict[header] = value
            elif not value_td:
                header = header_td.get_text(strip=True)
                stars = stars_td.get_text(strip=True)
                fill_stars = stars_td.find_all('span', class_='fill')
                review_dict[header] = len(fill_stars)

        author = article.find('div', {'class': 'body'}).find('h3').find('span', {'itemprop': 'author'})
        if author.a is not None:
            passenger_name = author.find('span', {'itemprop': 'name'}).text
            no_of_reviews = int(re.findall(get_int_pattern, author.a.span.text)[0])
        elif author.a is None:
            passenger_name = author.text.removeprefix('\n').removesuffix('\n').split('\n')[-1]
            no_of_reviews = 0
            
        # .find('strong').text
        text_content = article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'text_content'})
        if text_content.strong is not None:
            is_verified = text_content.find('strong').text
            detailed_review = text_content.text.split('|')[-1].strip(' ')
        elif text_content.strong is None:
            is_verified = 'not specified'
            detailed_review = text_content.text

        
        article_dict.append({
            'id': counts,
            'review_header': article.find('div', {'class': 'body'}).find('h2', {'class': 'text_header'}).text.strip('"'),
            'no_of_reviews': no_of_reviews,
            'passenger_name': passenger_name,
            'country_of_origin': ''.join(re.findall(pattern, article.find('div', {'class': 'body'}).find('h3').text)),
            'date': article.find('div', {'class': 'body'}).find('h3').find('time').text,
            'is_verified': is_verified,
            'detailed_review': detailed_review,
            'rating_dict': review_dict,
        })
    
        counts+=1

In [12]:
len(article_dict)

3642

In [17]:
from pprint import pprint
pprint(article_dict[100])

{'country_of_origin': 'United States',
 'date': '13th May 2023',
 'detailed_review': 'I booked business class tickets for my fiancé and I. I '
                    'had to reschedule my flight due to an unforeseen timing '
                    'change, so I called to get have my flight refunded. The '
                    "representative on the phone informed me he wasn't sure "
                    'how much I would get, as the refund department needed to '
                    'check, but estimated I would lose 10-20% of my booking to '
                    'fees. I accepted. I never got a cancellation email, or '
                    'any update, but instead received $35 total back. Less '
                    'than 1% of my booking costs. Since then I have spent over '
                    '5 hours calling different departments before they blocked '
                    "me. I've lost a tremendous amount of money with nothing "
                    'to show.',
 'id': 100,
 'is_verified': 'Tri

In [18]:
file_path = 'british_airways_reviews.json'

In [19]:
with open(file_path, 'w') as json_file:
    json.dump(article_dict, json_file)